## Import Packages

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import glob
import cv2
import collections

%matplotlib inline
np.set_printoptions(threshold=np.nan)

## Camera Calibration

In [2]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

nx = 9
ny = 6
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((ny*nx,3), np.float32)
objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (nx,ny), corners, ret)
        #write_name = 'corners_found'+str(idx)+'.jpg'
        #cv2.imwrite(write_name, img)
        cv2.imshow('img', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

In [3]:
import pickle

img = cv2.imread('test_images/test0_0.jpg')
img_size = (img.shape[1], img.shape[0])

def CameraCalibration(objpoints, imgpoints, img_size):

    # Do camera calibration given object points and image points
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
    
    return ret, mtx, dist, rvecs, tvecs

    # Save the camera calibration result for later use (we won't worry about rvecs / tvecs)

ret, mtx, dist, rvecs, tvecs = CameraCalibration(objpoints, imgpoints, img_size)    
    
dist_pickle = {}
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump(dist_pickle, open("camera_cal/camera_cal.p", "wb"))

## Distortion Correction

In [4]:
def DistortionCorrection(img, mtx, dist):
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    return dst

## Color and Gradient Threshold

In [5]:
def Abs_Sobel_Thresh(image, channel = 'gray', orient='x', sobel_kernel=3, thresh=(0, 255)):
    # Calculate directional gradient
    # Apply threshold
    if channel == 'gray':
        img = Grayscale(image)
    elif channel == 'sat':
        img = S_Channel(image)
        
    if orient == 'x':
        sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0)
        abs_sobel = np.absolute(sobelx)
    elif orient == 'y':
        sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1)
        abs_sobel = np.absolute(sobely)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return grad_binary

def Mag_Thresh_Binary(image, channel = 'gray', sobel_kernel=3, mag_thresh=(0, 255)):
    # Calculate gradient magnitude
    # Apply threshold
    if channel == 'gray':
        img = Grayscale(image)
    elif channel == 'sat':
        img = S_Channel(image)
        
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0 , 1, ksize=sobel_kernel)
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    scale_factor = np.max(gradmag)/255
    gradmag = (gradmag/scale_factor).astype(np.uint8)
    mag_binary = np.zeros_like(gradmag)
    mag_binary[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    return mag_binary

def Dir_Threshold(image, channel = 'gray', sobel_kernel=3, thresh=(0, np.pi/2)):
    # Calculate gradient direction
    # Apply threshold
    if channel == 'gray':
        img = Grayscale(image)
        
    elif channel == 'sat':
        img = S_Channel(image)
        
    sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
    sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize = sobel_kernel)
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    dirgrad = np.arctan2(abs_sobely, abs_sobelx)
    dir_binary = np.zeros_like(dirgrad)
    dir_binary[(dirgrad >= thresh[0]) & (dirgrad <= thresh[1])] = 1
    return dir_binary

def S_Channel(image):
    img = np.copy(image)
    hsv = cv2. cvtColor(image, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hsv[:,:,1]
    s_channel = hsv[:,:,2]
    
    return s_channel

def Grayscale(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    return gray
# Choose a Sobel kernel size
ksize = 3 # Choose a larger odd number to smooth gradient measurements

def Combined_Grad(image, sat, ksize, thresh, mag_thresh, dir_binary):
# Apply each of the thresholding functions
    gradx = Abs_Sobel_Thresh(image, channel = 'sat', orient='x', sobel_kernel=ksize, thresh=(50, 175))
    grady = Abs_Sobel_Thresh(image, channel = 'sat', orient='y', sobel_kernel=ksize, thresh=(50, 175))
    mag_binary = Mag_Thresh_Binary(image, channel = 'sat', sobel_kernel=ksize, mag_thresh=(50,175))
    dir_binary = Dir_Threshold(image, channel = 'sat', sobel_kernel=ksize, thresh=(0.7, 1.3))

    gradx_gray = Abs_Sobel_Thresh(image, channel = 'gray', orient='x', sobel_kernel=ksize, thresh=(50, 175))
    grady_gray = Abs_Sobel_Thresh(image, channel = 'gray', orient='y', sobel_kernel=ksize, thresh=(50, 175))
    mag_binary_gray = Mag_Thresh_Binary(image, channel = 'gray', sobel_kernel=ksize, mag_thresh=(50, 175))
    dir_binary_gray = Dir_Threshold(image, channel = 'gray', sobel_kernel=ksize, thresh=(0.7, 1.3))

    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1)) | ((gradx_gray == 1) & (grady_gray == 1)) | ((mag_binary_gray == 1) & (dir_binary_gray == 1))] = 1
    
    return combined

## Region of Interest Mask

In [6]:
def Region_Of_Interest(image, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(image)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(image.shape) > 2:
        channel_count = image.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image, mask


## Hough Transform

In [7]:
def Hough_Trans(image, rho, theta, threshold, min_line_len, max_line_gap, center, line_image):
    
    image = np.uint8(255*image/np.max(image))
    lines = cv2.HoughLinesP(image, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    l_lines_x = []
    r_lines_x = []
    l_lines_y = []
    r_lines_y = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
            if (x1 < center) & (x2 < center):
                l_lines_x.append(x1)
                l_lines_y.append(y1)
                l_lines_x.append(x2)
                l_lines_y.append(y2)
            elif (x1 > center) & (x2 > center):
                r_lines_x.append(x1)
                r_lines_y.append(y1)
                r_lines_x.append(x2)
                r_lines_y.append(y2)
    if len(l_lines_x)>0:            
        left_lane_line = np.polyfit(l_lines_x,l_lines_y,1) #left_lane_line = [m,b]
    else: 
        left_lane_line = [0,0]
    
    if len(r_lines_x)>0:
        right_lane_line = np.polyfit(r_lines_x,r_lines_y,1) #rightt_lane_line = [m,b]
    else: 
        right_lane_line = [0,0]
    
    return left_lane_line, right_lane_line, line_image

## Perspective Transform Variables

In [8]:
def Transform_Var():
    
    src = np.float32([(190, 720), (548, 480), (740, 480), (1130, 720)])
    dst = np.float32([(190, 720), (190, 0), (1130, 0), (1130, 720)])
        
    return src, dst

## Perspective Transform

In [9]:
def Corners_Warp(img, src, dst, mtx, dist, img_size):
    
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(img, M, img_size)

    return warped, M, Minv

## Histogram

In [10]:
def Find_Lines(top_down):
    histogram = np.sum(top_down[top_down.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((top_down, top_down, top_down))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 7
    # Set height of windows
    window_height = np.int(top_down.shape[0]/nwindows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = top_down.nonzero() # Return the indices of the elements that are non-zero.
    nonzeroy = np.array(nonzero[0]) #array of y-coordinates of non zero elements of top_down
    nonzerox = np.array(nonzero[1]) #corresponding array of x-coordinates of non zero elements of top_down
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        #Identify window boundaries in x and y (and right and left)
        win_y_low = top_down.shape[0] - (window+1)*window_height 
        win_y_high = top_down.shape[0] - window*window_height #closer to bottom of frame
        win_xleft_low = leftx_current - margin #left side of left box
        win_xleft_high = leftx_current + margin #right side of left box
        win_xright_low = rightx_current - margin #left side of right box
        win_xright_high = rightx_current + margin
        #Draw the windows on the visualization image
        cv2.rectangle(out_img, (win_xleft_low, win_y_low),(win_xleft_high, win_y_high),(0,255,0),5)
        cv2.rectangle(out_img, (win_xright_low, win_y_low),(win_xright_high, win_y_high),(0,255,0),5)
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    return left_fit, right_fit, leftx, lefty, rightx, righty, out_img

## Print Line of Best Fit with Left and Right Lane Pixels

In [11]:
def linefit(top_down, left_fit, right_fit):
    ploty = np.linspace(0, top_down.shape[0]-1, top_down.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    return ploty, left_fitx, right_fitx

## Skipping Sliding Windows Step and Searching in Region

In [12]:
def nextframe(top_down, left_fit, right_fit):
    nonzero = top_down.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100 #
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    if len(leftx)>10:
        left_fit = np.polyfit(lefty, leftx, 2)
    else:
        left_fit = [0,0,0]
        
    if len(rightx)>10:
        right_fit = np.polyfit(righty, rightx, 2)
    else:
        right_fit = [0,0,0]
    # Generate x and y values for plotting
    ploty = np.linspace(0, top_down.shape[0]-1, top_down.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    return left_fit, right_fit, ploty, left_fitx, right_fitx

## Measuring Curvature

In [13]:
def Curvature(ploty, leftx, rightx):    
    y_eval = np.max(ploty)

    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    #radius = (left_curverad+right_curverad)/2 
    #print(left_curverad, 'm', right_curverad, 'm')
    # Example values: 632.1 m    626.2 m
    
    return left_curverad, right_curverad 

## Measuring Offset

In [14]:
def Offset(top_down):
    
    histogram = np.sum(top_down[top_down.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((top_down, top_down, top_down))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    midlane = (leftx_base+rightx_base)/2
    midcar = (top_down.shape[1])/2
    px_2_m = 0.004111
    
    offset = px_2_m*(np.absolute(midlane-midcar))
    
    return offset

## Drawing

In [15]:
def Drawing(image, top_down, left_fitx, right_fitx, ploty, Minv):
# Create an image to draw the lines on
    warp_zero = np.zeros_like(top_down).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    newwarp = newwarp.astype(dtype = np.uint8)
    result = cv2.addWeighted(image, 1, newwarp, 0.3, 0)
    
    return result

## Line Class

In [16]:
class Line():
    def __init__(self, n=5):
        # number of previous frames to save line data from
        self.n = n
        # number of fits in buffer
        self.n_buffered = 0
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = collections.deque([], maxlen=n)
        # y values of the last n fits of the line
        self.recent_yfitted = collections.deque([], maxlen=n)
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float')                 

        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
        
        #x values of the last n line pixels
        self.recent_xplot = collections.deque([], maxlen=n)
        #y values of the last n line pixels
        self.recent_yplot = collections.deque([], maxlen=n)

## Make Radius and Offset Caption

In [17]:
def Caption(result, offset_dist, l_radius, r_radius):
    font = cv2.FONT_HERSHEY_SIMPLEX
    offset_str = str('offset from center: '+str(offset_dist*100)+'cm')
    cv2.putText(result,offset_str,(20,30), font, 1,(255,255,255),2,cv2.LINE_AA)
    if l_radius and r_radius:
        radius = round((l_radius/1000+r_radius/1000)/2,1) 
        str2 = str('radius of curvature: '+str(radius)+'km')
        cv2.putText(result,str2,(20,60), font, 1,(255,255,255),2,cv2.LINE_AA)    

## Averaging for Final Output 

In [18]:
def Average(self):
    fits = self.recent_xfitted
    if len(fits)>0:
        avg=0
        for fit in fits:
            avg +=np.array(fit)
        avg = avg / len(fits)
        
    return avg

## Process Image

In [19]:
def Process_Image(image):
    
    img_size = (image.shape[1], image.shape[0])
    
    distort = DistortionCorrection(image, mtx, dist) #UNDISTORTED IMAGE
    
    #Binary Image Parameters
    channel = 'sat'
    ksize = 3
    thresh = (50, 175)
    mag_thresh = (50, 175)
    dir_thresh = (0.7, 1.3)
    
    combined = Combined_Grad(distort, channel, ksize, thresh, mag_thresh, dir_thresh) #BINARY IMAGE
    
    src, dst = Transform_Var()
    
    #Hough Transform Parameters
    rho = 1
    theta = np.pi/180
    threshold = 50
    min_line_len = 30
    max_line_gap = 40
    line_image = np.copy(combined)*0
    center = line_image.shape[1]/2
    
    if left.detected & right.detected: 
        
        top_down, M, Minv = Corners_Warp(combined, src, dst, mtx, dist, img_size)
        left_fit, right_fit, ploty, left_fitx, right_fitx = nextframe(top_down, left.current_fit, right.current_fit)
        
        if left_fit[0] == 0:
            left_fit = left.current_fit
            left_fitx = left.allx
            left.detected = False
        else:
            left.detected = True
            
        if right_fit[0] == 0:
            right_fit = right.current_fit
            right_fitx = right.allx
            right.detected = False
        else:
            right.detected = True
        
    else:
        
        len_y = img_size[1]
        len_x = img_size[0]
        vertices = np.array([[(0,len_y), (len_x/2,425), (len_x,len_y)]], dtype=np.int32)
        masked_image, mask = Region_Of_Interest(combined, vertices)
        left_lane_line, right_lane_line, line_image = Hough_Trans(masked_image, rho, theta, threshold, min_line_len, max_line_gap, center, line_image)
        
        if (left_lane_line[0] != 0) & (right_lane_line[0] != 0):
            
            top_down, M, Minv = Corners_Warp(combined, src, dst, mtx, dist, img_size)
            left_fit, right_fit, leftx, lefty, rightx, righty, out_img = Find_Lines(top_down)
            ploty, left_fitx, right_fitx = linefit(top_down, left_fit, right_fit)
            left.detected = True
            right.detected = True
            
        else:
            
            top_down, M, Minv = Corners_Warp(combined, src, dst, mtx, dist, img_size)
            left_fitx = left.allx
            right_fitx = right.allx
            left_fit = left.current_fit
            right_fit = right.current_fit
            ploty = np.linspace(0, top_down.shape[0]-1, top_down.shape[0])
            left.detected = False
            right.detected = False
            
    left.diffs = np.absolute(left_fit - left.current_fit)
    right.diffs = np.absolute(right_fit - right.current_fit)
    
    left.current_fit = left_fit
    right.current_fit = right_fit
    
    left.recent_xfitted.append(left_fit)
    right.recent_xfitted.append(right_fit)
    
    left.allx = left_fitx
    right.allx = right_fitx
    
    left.ally = ploty
    right.ally = ploty
    
    left.recent_xplot.append(left_fitx)
    right.recent_xplot.append(right_fitx)
    
    left.recent_yplot.append(ploty)
    right.recent_yplot.append(ploty)
    
    #Averaged Values of Fit, and Fitx
    
    if len(left.recent_xfitted)>0:
        left_fit_avg = Average(left)
    else:
        left_fit_avg = left_fit
    
    if len(right.recent_xfitted)>0:
        right_fit_avg = Average(right)
    else:
        right_fit_avg = right_fit
        
    #Plotted Fit Lines
    
    ploty, left_fitx_avg, right_fitx_avg = linefit(top_down, left_fit_avg, right_fit_avg)
    
    #Drawn Lines of Image
    
    result = Drawing(image, top_down, left_fitx_avg, right_fitx_avg, ploty, Minv)
    
    leftcurve, rightcurve = Curvature(ploty, left_fitx_avg, right_fitx_avg)
    offset = Offset(top_down)
    
    Caption(result, offset, leftcurve, rightcurve)
    
    return result
    
    
    



## Make Video

In [20]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [1]:
def Video():
    white_output = 'test_videos_output/project_video.mp4'
    ## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
    ## To do so add .subclip(start_second,end_second) to the end of the line below
    ## Where start_second and end_second are integer values representing the start and end of the subclip
    ## You may also uncomment the following line for a subclip of the first 5 seconds
    ##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
    clip1 = VideoFileClip("project_video.mp4")
    white_clip = clip1.fl_image(Process_Image) #NOTE: this function expects color images!!
    %time white_clip.write_videofile(white_output, audio=False)

## Pipeline

In [23]:
left = Line()
right = Line()
Video()

[MoviePy] >>>> Building video test_videos_output/challenge_video_output.mp4
[MoviePy] Writing video test_videos_output/challenge_video_output.mp4


100%|█████████████████████████████████████▉| 1260/1261 [09:23<00:00,  2.25it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/challenge_video_output.mp4 

Wall time: 9min 23s
